## Start

Mount the Drive and install dependeces.

In [ ]:
## Examples: https://github.com/leimao/Frozen-Graph-TensorFlow/tree/master/TensorFlow_v2
## OpenCV: https://github.com/opencv/opencv/issues/16879
## Load: https://stackoverflow.com/questions/57093345/how-to-add-custom-keras-model-in-opencv-in-python

In [ ]:
!pip install tensorflow==1.15.0 tensorflow-gpu==1.15.0 keras==2.1.0

In [ ]:
# Antes de executar, conferir se o ambiente está configurado como GPU
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Modelo 3



In [ ]:
import tensorflow as tf
import os
import sys

# Root directory of the project
ROOT_DIR = '/content/drive/MyDrive/Image_segmentation/Mask_RCNN'

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import utils
import keras.backend as K

##############################################################################
# Load model
##############################################################################


# Model Directory
DEFAULT_WEIGHTS = '/content/drive/MyDrive/Image_segmentation/Mask_RCNN/mask_rcnn_coco.h5'
# Download COCO trained weights from Releases if needed
if not os.path.exists(DEFAULT_WEIGHTS):
    utils.download_trained_weights(DEFAULT_WEIGHTS)

##############################################################################
# Load configuration
##############################################################################

class InferenceConfig(Config):
        # Set batch size to 1 since we'll be running inference on
        # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
        DETECTION_MIN_CONFIDENCE = 0
        NAME = 'pvc_teste'
        NUM_CLASSES = 1 + 80 

sess = tf.Session()
K.set_session(sess)

# if __name__ == "__main__":
config = InferenceConfig()
config.display()

DEFAULT_LOGS_DIR = '/content/drive/MyDrive/Image_segmentation/Mask_RCNN/logs'

# Create model in inference mode
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=DEFAULT_LOGS_DIR)

cpu = 'y'#input('Make file to run on cpu? (y/n)')
if cpu == 'y':
    DEVICE = "/cpu:0"
    with tf.device(DEVICE):
        model = modellib.MaskRCNN(
            mode="inference", config=config, model_dir=DEFAULT_LOGS_DIR)

# Set path to model weights
weights_path = DEFAULT_WEIGHTS#model.find_last()
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)
model.keras_model.summary()

model_dir = '/content/drive/MyDrive/Image_segmentation/models/tf_model'
# make folder for full model
# model_dir = os.path.join(ROOT_DIR, "Model")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# import io
# from tensorflow.python.tools import freeze_graph
# from tensorflow.python.tools import optimize_for_inference_lib

MODEL_NAME = 'mask_rcnn_test.pb'

# model_output_dir = '/content/drive/MyDrive/Image_segmentation/models/tf_model'
input_node_names = [node.op.name for node in model.keras_model.outputs]
output_node_name = [node.op.name for node in model.keras_model.outputs]

min_graph = tf.compat.v1.graph_util.convert_variables_to_constants(sess, sess.graph_def, output_node_name)
tf.compat.v1.train.write_graph(min_graph, model_dir, MODEL_NAME, as_text=False)

# output_graph_def = optimize_for_inference_lib.optimize_for_inference(
#             min_graph, input_node_names, output_node_name,
#             tf.float32.as_datatype_enum)

# optimized_graph_file = os.path.join(model_output_dir, 'optimized_' + MODEL_NAME)
# with tf.gfile.GFile(optimized_graph_file, "wb") as f:
#     f.write(output_graph_def.SerializeToString())


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE           

'/content/drive/MyDrive/Image_segmentation/models/tf_model/mask_rcnn_test'

In [ ]:
output = [node.op.name for node in model.keras_model.outputs]
print(output)

['mrcnn_detection_1/Reshape_1', 'mrcnn_class_1/Reshape_1', 'mrcnn_bbox_1/Reshape', 'mrcnn_mask_1/Reshape_1', 'ROI_1/packed_2', 'rpn_class_1/concat', 'rpn_bbox_1/concat']


In [ ]:
input = [node.op.name for node in model.keras_model.inputs]
print(input)

['input_image_15', 'input_image_meta_15', 'input_anchors_15']


## Modelo 6 - Export Model

In [ ]:
# def export_model(model_output_dir, input_node_names, output_node_name):
import argparse
import io
import os

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

MODEL_NAME = 'mask_rcnn_test'

model_output_dir = '/content/drive/MyDrive/Image_segmentation/models/tf_model'
input_node_names = [node.op.name for node in model.keras_model.outputs]
output_node_name = [node.op.name for node in model.keras_model.outputs]

saver = tf.train.Saver()

with tf.Session() as sess:
    # Initialize the variables.
    sess.run(tf.global_variables_initializer())

    # Save the graph definition to a file.
    tf.train.write_graph(sess.graph_def, model_output_dir,
                          MODEL_NAME + '.pbtxt', True)

    checkpoint_file = os.path.join(model_output_dir, MODEL_NAME + '.ckpt')
    # for step in range (200):
    #     # Every 10,000 iterations, we save a checkpoint of the model.
    #     if step % 100 == 0:
    #         saver.save(sess, checkpoint_file, global_step=step)

    # Save a checkpoint after training has completed.
    ckpt_path = saver.save(sess, checkpoint_file)

    # ---------------------------------------------------
    """Export the model so we can use it later.

    This will create two Protocol Buffer files in the model output directory.
    These files represent a serialized version of our model with all the
    learned weights and biases. One of the ProtoBuf files is a version
    optimized for inference-only usage.
    """
    name_base = os.path.join(model_output_dir, MODEL_NAME)
    frozen_graph_file = os.path.join(model_output_dir,
                                      'frozen_' + MODEL_NAME + '.pb')

    freeze_graph.freeze_graph(
        name_base + '.pbtxt', None, False, ckpt_path,
        output_node_name, "save/restore_all", "save/Const:0",
        frozen_graph_file, True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open(frozen_graph_file, "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    optimized_graph_file = os.path.join(model_output_dir,
                                        'optimized_' + MODEL_NAME + '.pb')
    with tf.gfile.GFile(optimized_graph_file, "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("Inference optimized graph saved at: " + optimized_graph_file)

    sess.close()

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Image_segmentation/models/tf_model/mask_rcnn_test.ckpt


AttributeError: ignored

## TF2.X

In [ ]:
print(tf.__version__)

from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
# tf.config.run_functions_eagerly(True)

f = tf.function(lambda x: model.keras_model(x))
f = tf.function(f).get_concrete_function(x=(tf.TensorSpec(model.keras_model.inputs[0].shape, model.keras_model.inputs[0].dtype),
                                                            tf.TensorSpec(model.keras_model.inputs[1].shape, model.keras_model.inputs[1].dtype),
                                                            tf.TensorSpec(model.keras_model.inputs[2].shape, model.keras_model.inputs[2].dtype)),)
print(tf.executing_eagerly())
f2 = convert_variables_to_constants_v2(f)
graph_def = f2.graph.as_graph_def()

# Export frozen graph
with tf.io.gfile.GFile('/content/drive/MyDrive/Image_segmentation/models/frozen_model/frozen_graph.pb', 'wb') as f:
   f.write(graph_def.SerializeToString())

2.4.1
True


In [ ]:
#To make .pbtxt file
https://github.com/opencv/opencv/blob/master/samples/dnn/tf_text_graph_ssd.py

## Modelo 4 - Lei Mao

Model to freeze a graph from [Lei Mao][] github.

[Lei Mao]: https://leimao.github.io/blog/Save-Load-Inference-From-TF-Frozen-Graph/

In [ ]:
import tensorflow as tf
import os

from tensorflow.python.tools import freeze_graph
from tensorflow.python.framework import graph_util

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model import utils as saved_model_utils


class CNN(object):
    def __init__(self, input_size, num_classes, optimizer):

        self.num_classes = num_classes
        self.input_size = input_size
        self.optimizer = optimizer

        self.learning_rate = tf.placeholder(tf.float32,
                                            shape=[],
                                            name='learning_rate')
        self.dropout_rate = tf.placeholder(tf.float32,
                                           shape=[],
                                           name='dropout_rate')
        self.input = tf.placeholder(tf.float32, [None] + self.input_size,
                                    name='input')
        self.label = tf.placeholder(tf.float32, [None, self.num_classes],
                                    name='label')
        self.output = self.network_initializer()
        self.loss = self.loss_initializer()
        self.optimization = self.optimizer_initializer()

        self.saver = tf.train.Saver()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def network(self, input, dropout_rate):

        conv1 = tf.layers.conv2d(inputs=input,
                                 filters=64,
                                 kernel_size=[3, 3],
                                 padding='same',
                                 activation=tf.nn.relu,
                                 name='conv1')

        conv2 = tf.layers.conv2d(inputs=conv1,
                                 filters=64,
                                 kernel_size=[3, 3],
                                 padding='same',
                                 activation=tf.nn.relu,
                                 name='conv2')

        pool1 = tf.layers.max_pooling2d(inputs=conv2,
                                        pool_size=[2, 2],
                                        strides=[2, 2],
                                        name='pool1')

        pool1_dropout = tf.layers.dropout(inputs=pool1,
                                          rate=dropout_rate,
                                          training=True,
                                          name='pool1_dropout')

        conv3 = tf.layers.conv2d(inputs=pool1_dropout,
                                 filters=128,
                                 kernel_size=[3, 3],
                                 padding='same',
                                 activation=tf.nn.relu,
                                 name='conv3')

        conv4 = tf.layers.conv2d(inputs=conv3,
                                 filters=128,
                                 kernel_size=[3, 3],
                                 padding='same',
                                 activation=tf.nn.relu,
                                 name='conv4')

        pool2 = tf.layers.max_pooling2d(inputs=conv4,
                                        pool_size=[2, 2],
                                        strides=[2, 2],
                                        name='pool2')

        pool2_dropout = tf.layers.dropout(inputs=pool2,
                                          rate=dropout_rate,
                                          training=True,
                                          name='pool2_dropout')

        conv5 = tf.layers.conv2d(inputs=pool2_dropout,
                                 filters=256,
                                 kernel_size=[3, 3],
                                 padding='same',
                                 activation=tf.nn.relu,
                                 name='conv5')

        pool3 = tf.layers.max_pooling2d(inputs=conv5,
                                        pool_size=[2, 2],
                                        strides=[2, 2],
                                        name='pool3')

        pool3_dropout = tf.layers.dropout(inputs=pool3,
                                          rate=dropout_rate,
                                          training=True,
                                          name='pool3_dropout')

        flat = tf.layers.flatten(inputs=pool3_dropout, name='flat')

        fc1 = tf.layers.dense(inputs=flat,
                              units=256,
                              activation=tf.nn.relu,
                              name='fc1')

        fc1_dropout = tf.layers.dropout(inputs=fc1,
                                        rate=dropout_rate,
                                        training=True,
                                        name='fc1_dropout')

        fc2 = tf.layers.dense(inputs=fc1_dropout,
                              units=self.num_classes,
                              activation=None,
                              name='fc2')

        # Give output node a
        output = tf.identity(fc2, name='output')

        return output

    def network_initializer(self):

        with tf.variable_scope('cnn') as scope:
            ouput = self.network(input=self.input,
                                 dropout_rate=self.dropout_rate)

        return ouput

    def loss_initializer(self):

        with tf.variable_scope('loss') as scope:
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
                labels=self.label, logits=self.output, name='cross_entropy')
            cross_entropy_mean = tf.reduce_mean(cross_entropy,
                                                name='cross_entropy_mean')
        return cross_entropy_mean

    def optimizer_initializer(self):

        if self.optimizer == 'Adam':
            optimizer = tf.train.AdamOptimizer(
                learning_rate=self.learning_rate).minimize(self.loss)
        else:
            optimizer = tf.train.GradientDescentOptimizer(
                learning_rate=self.learning_rate).minimize(self.loss)

        return optimizer

    def train(self, data, label, learning_rate, dropout_rate):

        _, train_loss = self.sess.run(
            [self.optimization, self.loss],
            feed_dict={
                self.input: data,
                self.label: label,
                self.learning_rate: learning_rate,
                self.dropout_rate: dropout_rate
            })
        return train_loss

    def validate(self, data, label):

        output, validate_loss = self.sess.run([self.output, self.loss],
                                              feed_dict={
                                                  self.input: data,
                                                  self.label: label,
                                                  self.dropout_rate: 0.0
                                              })
        return output, validate_loss

    def test(self, data):

        output = self.sess.run(self.output,
                               feed_dict={
                                   self.input: data,
                                   self.dropout_rate: 0.0
                               })

        return output

    def save(self, directory, filename):

        if not os.path.exists(directory):
            os.makedirs(directory)
        filepath = os.path.join(directory, filename + '.ckpt')
        self.saver.save(self.sess, filepath)
        return filepath

    def save_signature(self, directory):

        signature = signature_def_utils.build_signature_def(
            inputs={
                'input':
                saved_model_utils.build_tensor_info(self.input),
                'dropout_rate':
                saved_model_utils.build_tensor_info(self.dropout_rate)
            },
            outputs={
                'output': saved_model_utils.build_tensor_info(self.output)
            },
            method_name=signature_constants.PREDICT_METHOD_NAME)
        signature_map = {
            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature
        }
        model_builder = saved_model_builder.SavedModelBuilder(directory)
        model_builder.add_meta_graph_and_variables(
            self.sess,
            tags=[tag_constants.SERVING],
            signature_def_map=signature_map,
            clear_devices=True)
        model_builder.save(as_text=False)

    def save_as_pb(self, directory, filename):

        if not os.path.exists(directory):
            os.makedirs(directory)

        # Save check point for graph frozen later
        ckpt_filepath = self.save(directory=directory, filename=filename)
        pbtxt_filename = filename + '.pbtxt'
        pbtxt_filepath = os.path.join(directory, pbtxt_filename)
        pb_filepath = os.path.join(directory, filename + '.pb')
        # This will only save the graph but the variables will not be saved.
        # You have to freeze your model first.
        tf.train.write_graph(graph_or_graph_def=self.sess.graph_def,
                             logdir=directory,
                             name=pbtxt_filename,
                             as_text=True)

        # Freeze graph
        # Method 1
        freeze_graph.freeze_graph(input_graph=pbtxt_filepath,
                                  input_saver='',
                                  input_binary=False,
                                  input_checkpoint=ckpt_filepath,
                                  output_node_names='cnn/output',
                                  restore_op_name='save/restore_all',
                                  filename_tensor_name='save/Const:0',
                                  output_graph=pb_filepath,
                                  clear_devices=True,
                                  initializer_nodes='')

        # Method 2
        '''
        graph = tf.get_default_graph()
        input_graph_def = graph.as_graph_def()
        output_node_names = ['cnn/output']
        output_graph_def = graph_util.convert_variables_to_constants(self.sess, input_graph_def, output_node_names)
        with tf.gfile.GFile(pb_filepath, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        '''

        return pb_filepath

    def load(self, filepath):

        if os.path.splitext(filepath)[1] != '.ckpt':
            filepath += '.ckpt'

        self.saver.restore(self.sess, filepath)

## Modelo 5 - Freeze and Optimize

Freeze and Optimaze from [Program Creek][].

* [Code][]

[Program Creek]: https://www.programcreek.com/python/example/91439/tensorflow.python.tools.freeze_graph.freeze_graph
[Code]: https://www.programcreek.com/python/?code=PacktPublishing%2FIntelligent-Projects-Using-Python%2FIntelligent-Projects-Using-Python-master%2FChapter07%2Ffreeze_code.py

In [ ]:
import sys
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib
import fire
from elapsedtimer import ElapsedTimer

path = '/home/santanu/Downloads/Mobile_App/'
MODEL_NAME = 'model'

# def model_freeze(path,MODEL_NAME='model'):

# Freeze the graph

input_graph_path = path + MODEL_NAME+'.pbtxt'
checkpoint_path = path + 'model_ckpt'
input_saver_def_path = ""
input_binary = False
output_node_names = 'positive_sentiment_probability'
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = path + 'frozen_'+MODEL_NAME+'.pb'
output_optimized_graph_name = path + 'optimized_'+MODEL_NAME+'.pb'
clear_devices = True


freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                        input_binary, checkpoint_path, output_node_names,
                        restore_op_name, filename_tensor_name,
output_frozen_graph_name, clear_devices, "")

input_graph_def = tf.GraphDef()

with tf.gfile.Open(output_frozen_graph_name, "rb") as f:
    data = f.read()
    input_graph_def.ParseFromString(data)

output_graph_def = optimize_for_inference_lib.optimize_for_inference(
        input_graph_def,
        ["inputs/X" ],#an array of the input node(s)
        ["positive_sentiment_probability"],
        tf.int32.as_datatype_enum # an array of output nodes
        )

# Save the optimized graph

f = tf.gfile.FastGFile(output_optimized_graph_name, "w")
f.write(output_graph_def.SerializeToString())

## Opencv - How to build .pbtxt file

* [Opencv][] tutorial to build .pbtxt file from a .pb model.

* [Issue][] from Opencv GitHub.

[Opencv]: https://github.com/opencv/opencv/wiki/TensorFlow-text-graphs
[Issue]: https://github.com/opencv/opencv/issues/19226

In [ ]:
import tensorflow as tf

pb_file = model_dir + '/' + MODEL_NAME
graph_def = tf.compat.v1.GraphDef()
pbtxt_path = model_dir + '/' + 'model.pbtxt'

try:
    with tf.io.gfile.GFile(pb_file, 'rb') as f:
        graph_def.ParseFromString(f.read())
except:
    with tf.gfile.FastGFile(pb_file, 'rb') as f:
        graph_def.ParseFromString(f.read())

# Delete weights
for i in reversed(range(len(graph_def.node))):
    if graph_def.node[i].op == 'Const':
        del graph_def.node[i]

graph_def.library.Clear()

tf.compat.v1.train.write_graph(graph_def, "", pbtxt_path, as_text=True)

'/content/drive/MyDrive/Image_segmentation/models/tf_model/model.pbtxt'

In [ ]:
import cv2 as cv

net = cv.dnn.readNetFromTensorflow(pb_file, pbtxt_path)

error: ignored

## Modelo 6 - Completo

Another Program Creek file.
* [Code][]

[Code]: https://www.programcreek.com/python/?code=IBM%2Ftensorflow-hangul-recognition%2Ftensorflow-hangul-recognition-master%2Fhangul_model.py

In [ ]:
#!/usr/bin/env python

import argparse
import io
import os

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib


SCRIPT_PATH = os.path.dirname(os.path.abspath(__file__))

# Default paths.
DEFAULT_LABEL_FILE = os.path.join(SCRIPT_PATH,
                                  './labels/2350-common-hangul.txt')
DEFAULT_TFRECORDS_DIR = os.path.join(SCRIPT_PATH, 'tfrecords-output')
DEFAULT_OUTPUT_DIR = os.path.join(SCRIPT_PATH, 'saved-model')

MODEL_NAME = 'hangul_tensorflow'
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64

DEFAULT_NUM_EPOCHS = 15
BATCH_SIZE = 100

# This will be determined by the number of entries in the given label file.
num_classes = 2350


def _parse_function(example):
    features = tf.parse_single_example(
        example,
        features={
            'image/class/label': tf.FixedLenFeature([], tf.int64),
            'image/encoded': tf.FixedLenFeature([], dtype=tf.string,
                                                default_value='')
        })
    label = features['image/class/label']
    image_encoded = features['image/encoded']

    # Decode the JPEG.
    image = tf.image.decode_jpeg(image_encoded, channels=1)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.reshape(image, [IMAGE_WIDTH*IMAGE_HEIGHT])

    # Represent the label as a one hot vector.
    label = tf.stack(tf.one_hot(label, num_classes))
    return image, label

def weight_variable(shape):
    """Generates a weight variable of a given shape."""
    initial = tf.random.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name='weight')


def bias_variable(shape):
    """Generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name='bias')


def main(label_file, tfrecords_dir, model_output_dir, num_train_epochs):
    """Perform graph definition and model training.

    Here we will first create our input pipeline for reading in TFRecords
    files and producing random batches of images and labels.
    Next, a convolutional neural network is defined, and training is performed.
    After training, the model is exported to be used in applications.
    """
    global num_classes
    labels = io.open(label_file, 'r', encoding='utf-8').read().splitlines()
    num_classes = len(labels)

    # Define names so we can later reference specific nodes for when we use
    # the model for inference later.
    input_node_name = 'input'
    keep_prob_node_name = 'keep_prob'
    output_node_name = 'output'

    if not os.path.exists(model_output_dir):
        os.makedirs(model_output_dir)

    print('Processing data...')

    tf_record_pattern = os.path.join(tfrecords_dir, '%s-*' % 'train')
    train_data_files = tf.gfile.Glob(tf_record_pattern)

    tf_record_pattern = os.path.join(tfrecords_dir, '%s-*' % 'test')
    test_data_files = tf.gfile.Glob(tf_record_pattern)

    # Create training dataset input pipeline.
    train_dataset = tf.data.TFRecordDataset(train_data_files) \
        .map(_parse_function) \
        .shuffle(1000) \
        .repeat(num_train_epochs) \
        .batch(BATCH_SIZE) \
        .prefetch(1)

    # Create the model!

    # Placeholder to feed in image data.
    x = tf.placeholder(tf.float32, [None, IMAGE_WIDTH*IMAGE_HEIGHT],
                       name=input_node_name)
    # Placeholder to feed in label data. Labels are represented as one_hot
    # vectors.
    y_ = tf.placeholder(tf.float32, [None, num_classes])

    # Reshape the image back into two dimensions so we can perform convolution.
    x_image = tf.reshape(x, [-1, IMAGE_WIDTH, IMAGE_HEIGHT, 1])

    # First convolutional layer. 32 feature maps.
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    x_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1],
                           padding='SAME')
    h_conv1 = tf.nn.relu(x_conv1 + b_conv1)

    # Max-pooling.
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1], padding='SAME')

    # Second convolutional layer. 64 feature maps.
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    x_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1],
                           padding='SAME')
    h_conv2 = tf.nn.relu(x_conv2 + b_conv2)

    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1], padding='SAME')

    # Third convolutional layer. 128 feature maps.
    W_conv3 = weight_variable([3, 3, 64, 128])
    b_conv3 = bias_variable([128])
    x_conv3 = tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1],
                           padding='SAME')
    h_conv3 = tf.nn.relu(x_conv3 + b_conv3)

    h_pool3 = tf.nn.max_pool(h_conv3, ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1], padding='SAME')

    # Fully connected layer. Here we choose to have 1024 neurons in this layer.
    h_pool_flat = tf.reshape(h_pool3, [-1, 8*8*128])
    W_fc1 = weight_variable([8*8*128, 1024])
    b_fc1 = bias_variable([1024])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, W_fc1) + b_fc1)

    # Dropout layer. This helps fight overfitting.
    keep_prob = tf.placeholder(tf.float32, name=keep_prob_node_name)
    h_fc1_drop = tf.nn.dropout(h_fc1, rate=1-keep_prob)

    # Classification layer.
    W_fc2 = weight_variable([1024, num_classes])
    b_fc2 = bias_variable([num_classes])
    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    # This isn't used for training, but for when using the saved model.
    tf.nn.softmax(y, name=output_node_name)

    # Define our loss.
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=tf.stop_gradient(y_),
            logits=y
        )
    )

    # Define our optimizer for minimizing our loss. Here we choose a learning
    # rate of 0.0001 with AdamOptimizer. This utilizes someting
    # called the Adam algorithm, and utilizes adaptive learning rates and
    # momentum to get past saddle points.
    train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

    # Define accuracy.
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)

    saver = tf.train.Saver()

    with tf.Session() as sess:
        # Initialize the variables.
        sess.run(tf.global_variables_initializer())

        checkpoint_file = os.path.join(model_output_dir, MODEL_NAME + '.chkp')

        # Save the graph definition to a file.
        tf.train.write_graph(sess.graph_def, model_output_dir,
                             MODEL_NAME + '.pbtxt', True)

        try:
            iterator = train_dataset.make_one_shot_iterator()
            batch = iterator.get_next()
            step = 0

            while True:

                # Get a batch of images and their corresponding labels.
                train_images, train_labels = sess.run(batch)

                # Perform the training step, feeding in the batches.
                sess.run(train_step, feed_dict={x: train_images,
                                                y_: train_labels,
                                                keep_prob: 0.5})
                if step % 100 == 0:
                    train_accuracy = sess.run(
                        accuracy,
                        feed_dict={x: train_images, y_: train_labels,
                                   keep_prob: 1.0}
                    )
                    print("Step %d, Training Accuracy %g" %
                          (step, float(train_accuracy)))

                # Every 10,000 iterations, we save a checkpoint of the model.
                if step % 10000 == 0:
                    saver.save(sess, checkpoint_file, global_step=step)

                step += 1

        except tf.errors.OutOfRangeError:
            pass

        # Save a checkpoint after training has completed.
        saver.save(sess, checkpoint_file)

        # See how model did by running the testing set through the model.
        print('Testing model...')

        # Create testing dataset input pipeline.
        test_dataset = tf.data.TFRecordDataset(test_data_files) \
            .map(_parse_function) \
            .batch(BATCH_SIZE) \
            .prefetch(1)

        # Define a different tensor operation for summing the correct
        # predictions.
        accuracy2 = tf.reduce_sum(correct_prediction)
        total_correct_preds = 0
        total_preds = 0

        try:
            iterator = test_dataset.make_one_shot_iterator()
            batch = iterator.get_next()
            while True:
                test_images, test_labels = sess.run(batch)
                acc = sess.run(accuracy2, feed_dict={x: test_images,
                                                     y_: test_labels,
                                                     keep_prob: 1.0})
                total_preds += len(test_images)
                total_correct_preds += acc

        except tf.errors.OutOfRangeError:
            pass

        test_accuracy = total_correct_preds/total_preds
        print("Testing Accuracy {}".format(test_accuracy))

        export_model(model_output_dir, [input_node_name, keep_prob_node_name],
                     output_node_name)

        sess.close()


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--label-file', type=str, dest='label_file',
                        default=DEFAULT_LABEL_FILE,
                        help='File containing newline delimited labels.')
    parser.add_argument('--tfrecords-dir', type=str, dest='tfrecords_dir',
                        default=DEFAULT_TFRECORDS_DIR,
                        help='Directory of TFRecords files.')
    parser.add_argument('--output-dir', type=str, dest='output_dir',
                        default=DEFAULT_OUTPUT_DIR,
                        help='Output directory to store saved model files.')
    parser.add_argument('--num-train-epochs', type=int,
                        dest='num_train_epochs',
                        default=DEFAULT_NUM_EPOCHS,
                        help='Number of times to iterate over all of the '
                             'training data.')
    args = parser.parse_args()
    main(args.label_file, args.tfrecords_dir,
         args.output_dir, args.num_train_epochs)